In [1]:
# !pip install datasets
!pip install transformers[torch]
# !pip install torchaudio
# !pip install librosa
# !pip install jiwer
# !pip install evaluate
!pip install accelerate

In [2]:
import os
from datasets import load_dataset, Audio

data_files = {
    "train": "/home/mehedi/Projects/Learning/wav2vec/local_bn_dataset_2/metadata_train.csv",
    "validation": "/home/mehedi/Projects/Learning/wav2vec/local_bn_dataset_2/metadata_validation.csv",
    "test": "/home/mehedi/Projects/Learning/wav2vec/local_bn_dataset_2/metadata_test.csv"
}

audio_base_path = "/home/mehedi/Projects/Learning/wav2vec/local_bn_dataset_2/"

my_local_dataset_train = load_dataset(
    "csv", 
    data_files={"train": data_files["train"], "validation": data_files["validation"]}, 
    split="train+validation"
)

my_local_dataset_test = load_dataset(
    "csv", 
    data_files={"test": data_files["test"]}, 
    split="test"
)

def add_absolute_path(example):
    # Create a new 'audio' column with the absolute path
    example["audio"] = os.path.join(audio_base_path, example["path"])
    return example

my_local_dataset_train = my_local_dataset_train.map(add_absolute_path)
my_local_dataset_test = my_local_dataset_test.map(add_absolute_path)

common_voice_train = my_local_dataset_train.cast_column("audio", Audio(sampling_rate=16000))
common_voice_test = my_local_dataset_test.cast_column("audio", Audio(sampling_rate=16000))



/home/mehedi/anaconda3/envs/wav2vec/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
my_local_dataset_train[0]

{'path': 'train/common_voice_bn_35453419.mp3',
 'sentence': 'প্রমার সভাপতি রাশেদ হাসানের সভাপতিত্বে অনুষ্ঠানে বিশেষ অতিথি ছিলেন কবি ওমর কায়সার।',
 'audio': '/home/mehedi/Projects/Learning/wav2vec/local_bn_dataset_2/train/common_voice_bn_35453419.mp3'}

In [5]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]), num_examples=10)

,sentence
0,এ সময়ে তিনি নিজের কবিতা প্রকাশ করতে পারতেন না।
1,এটা তখনকার সময়কার খুবই আধুনিক একটি দালান ছিল।
2,এই প্রত্যেকটি ভূমিরূপ কে আবার অনেক ভাগে ভাগ করা হয়েছে।
3,ভারত সরকারের পুরাতাত্ত্বিক বিভাগ এর দেখাশুনা করে।
4,পিতা চিকিৎসক সুরেন্দ্রনাথ দাশ ও মাতা বিশিষ্ট শিক্ষিকা সবিতা দাশ।
5,এটি একটি আশ্চর্যজনক প্রশিক্ষণ কর্মক্ষমতা।
6,"পরবর্তীতে, তিনি আরও দুটি কিডস চয়েস অ্যাওয়ার্ডস জিতে নেন এবং একটি পিপলস চয়েস অ্যাওয়ার্ড জিতেন।"
7,এটি পরিচালনা করেছেন মার্টিন স্কোরসেজি এবং চিত্রনাট্য লিখেছেন জন লোগান।
8,এটি উত্তর দিনাজপুর জেলার সদর দপ্তর।
9,তিনি তার শৈশব কাটান নতুন দিল্লিতে।


In [7]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [8]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

In [9]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence
0,তিনি একবার পশ্চিমবঙ্গ বিধানসভার সদস্য হিসেবে নির্বাচিত হয়েছেন।
1,কোয়ান্টাম মেকানিক্স।
2,বৌদ্ধধর্মে আদিবুদ্ধ শীর্ষক এ আস্তিক্যবাদী ধারণার উৎপত্তিস্থল সম্পর্কে মতভেদ আছে।
3,পরবর্তীতে তিনি লখনউ এ দরিদ্র ছাত্রছাত্রীদের জন্য একটি স্কুল প্রতিষ্ঠা করেন।
4,তিনি ছিলেন জমিদার ও বিত্তবান পরিবারের একমাত্র সন্তান।
5,এরপর তিনি দ্যা আফ্রিকান রিভিউ পত্রিকার ফিচার সম্পাদক ও গার্ডিয়ান টাইমসে ফ্রিল্যান্স লেখক হিসেবে কাজ করেন।
6,তার প্রাতিষ্ঠানিক শিক্ষা এইচএসসি পর্যন্ত।
7,ইসরায়েলে যেখানে পরিবারের অনেক সদস্য থাকেন তিনি এবং তার স্বামী একটি প্রযুক্তিগত কলেজে ফেলোশিপ দিয়েছিলেন।
8,ডানহাতি মিডিয়ামফাস্ট বোলার হিসেবে ভারত ক্রিকেট দলে খেলছেন।
9,কেন্দ্রটি ন্যাশনাল বোটানিক্যাল গার্ডেন নামেও পরিচিত।


In [10]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [11]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

Map: 100%|██████████| 1000/1000 [00:00<00:00, 199804.88 examples/s]


In [12]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [13]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 0,
 'হ': 1,
 'জ': 2,
 'ং': 3,
 'a': 4,
 'উ': 5,
 'অ': 6,
 'ঝ': 7,
 'ঔ': 8,
 'ট': 9,
 'ঊ': 10,
 '্': 11,
 'র': 12,
 'ৃ': 13,
 'য়': 14,
 'চ': 15,
 'শ': 16,
 'ত': 17,
 'ঋ': 18,
 'ন': 19,
 'ঢ': 20,
 '।': 21,
 'ফ': 22,
 '’': 23,
 'ৌ': 24,
 "'": 25,
 'ঈ': 26,
 'প': 27,
 'ঐ': 28,
 'ৎ': 29,
 'ঞ': 30,
 '—': 31,
 'l': 32,
 'ও': 33,
 'ঃ': 34,
 'ু': 35,
 'স': 36,
 'ঠ': 37,
 'ষ': 38,
 'o': 39,
 'ূ': 40,
 'ভ': 41,
 '়': 42,
 'e': 43,
 'খ': 44,
 'y': 45,
 'g': 46,
 '॥': 47,
 'ঘ': 48,
 '–': 49,
 'ম': 50,
 'p': 51,
 'ড়': 52,
 'ই': 53,
 'ণ': 54,
 'ি': 55,
 'ী': 56,
 'গ': 57,
 '/': 58,
 'া': 59,
 'ো': 60,
 'থ': 61,
 'য': 62,
 'ব': 63,
 'ঙ': 64,
 'এ': 65,
 'আ': 66,
 'ঢ়': 67,
 'ল': 68,
 'দ': 69,
 'ৈ': 70,
 'ছ': 71,
 'ক': 72,
 'ধ': 73,
 'ড': 74,
 'ে': 75,
 'ঁ': 76}

In [14]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [15]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

79

In [16]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [17]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [18]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [19]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [20]:
common_voice_train[0]["path"]

'train/common_voice_bn_35453419.mp3'

In [21]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [22]:
common_voice_train[0]["audio"]

{'path': '/home/mehedi/Projects/Learning/wav2vec/local_bn_dataset_2/train/common_voice_bn_35453419.mp3',
 'array': array([-2.32830644e-10, -4.65661287e-10, -4.65661287e-10, ...,
         1.02171907e-06,  5.25280484e-07,  3.05510912e-06]),
 'sampling_rate': 16000}

In [23]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

In [24]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: এতে তিনি ডাবল সেঞ্চুরি বা দ্বিশতক করেন। 
Input array shape: (92736,)
Sampling rate: 16000


In [25]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [26]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, num_proc=4)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, num_proc=4)

Map (num_proc=4):   0%|          | 0/9000 [00:00<?, ? examples/s]/home/mehedi/anaconda3/envs/wav2vec/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/mehedi/anaconda3/envs/wav2vec/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/mehedi/anaconda3/envs/wav2vec/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_process

In [27]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [28]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [29]:
import evaluate

# Load the Word Error Rate (WER) metric
wer_metric = evaluate.load("wer")

In [30]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [31]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",  
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
model.freeze_feature_extractor()

/home/mehedi/anaconda3/envs/wav2vec/lib/python3.10/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:2177: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [33]:
model.gradient_checkpointing_enable()

In [34]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./wav2vec2-large-xlsr-bn-demo",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=20,
  fp16=True,  # Disable mixed precision training on CPU
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

/home/mehedi/anaconda3/envs/wav2vec/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [35]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

/home/mehedi/anaconda3/envs/wav2vec/lib/python3.10/site-packages/accelerate/accelerator.py:449: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [36]:
trainer.train()

  0%|          | 0/5620 [00:00<?, ?it/s]/home/mehedi/anaconda3/envs/wav2vec/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/mehedi/anaconda3/envs/wav2vec/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
  0%|          | 10/5620 [00:39<5:30:26,  3.53s/it]

{'loss': 17.8706, 'grad_norm': 5.545180320739746, 'learning_rate': 5.399999999999999e-06, 'epoch': 0.04}


  0%|          | 20/5620 [01:09<4:22:51,  2.82s/it]

{'loss': 18.3859, 'grad_norm': 10.400908470153809, 'learning_rate': 1.14e-05, 'epoch': 0.07}


  1%|          | 30/5620 [01:33<3:30:06,  2.26s/it]

...

100%|██████████| 5620/5620 [4:24:30<00:00,  2.04s/it]

{'loss': 0.1424, 'grad_norm': 0.811058759689331, 'learning_rate': 1.7578124999999998e-07, 'epoch': 19.96}


100%|██████████| 5620/5620 [4:24:34<00:00,  2.82s/it]

{'train_runtime': 15874.0298, 'train_samples_per_second': 11.339, 'train_steps_per_second': 0.354, 'train_loss': 0.8742152554496752, 'epoch': 19.96}


TrainOutput(global_step=5620, training_loss=0.8742152554496752, metrics={'train_runtime': 15874.0298, 'train_samples_per_second': 11.339, 'train_steps_per_second': 0.354, 'total_flos': 3.271521370154527e+19, 'train_loss': 0.8742152554496752, 'epoch': 19.964476021314386})